Replace package.json latest with actual latest version numbers?



In [ ]:
var fs = require('fs');
var path = require('path');
var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\';
var execSync = require('child_process').execSync;
var packages = execSync('npm list --dev', {cwd: cwd}).toString().split('\n');
var packageJson = JSON.parse(fs.readFileSync(path.join(cwd, 'package.json')));
var deps = Object.keys(packageJson.dependencies);
deps.forEach((d, k) => {
    packages.forEach((p, i) => {
        if (p.indexOf(d + '@') > -1) {
            packageJson.dependencies[d] = '~' + p.split('@').pop().replace(' deduped', '');
        }
    });
});
console.log(packages);
var devDeps = Object.keys(packageJson.devDependencies);
devDeps.forEach((d, k) => {
    packages.forEach((p, i) => {
        if (p.indexOf(d + '@') > -1) {
            packageJson.devDependencies[d] = '~' + p.split('@').pop().replace(' deduped', '');
        }
    });
});
$$.mime({'text/html': '<pre>' + JSON.stringify(packageJson, null, 4) + '</pre>'});


Check if there is a local-npm server running on Brian's machine?



In [ ]:
var http = require('http');
var os = require('os');
function checkLocalNPM(host) {
    var HOST = host || os.hostname() || 'localhost';
    return http.get(
        'http://' + HOST + ':5080',
        (r) => r.statusCode === 200)
        .on('error', (e) => false);
};
module.exports = checkLocalNPM;


Run NPM in javscript with in-memory file-system?



In [1]:
var execSync = require('child_process').execSync;
try {
    require.resolve('npm');
} catch (e) {
    execSync('npm install npm');
}

function npm(project, args, conf) {
    var previous = process.cwd();
    process.chdir(project);
    return importer
        .import('memory-fs rewire')
        .then(r => {
            // set up project path
            return new Promise((resolve, reject) => {
                //if(typeof mockTypescriptFs == 'undefined') {
                //    mockTypescriptFs = eval("'use strict';" + r[0].code);
                // overlay out temp filesystem on top of current filesystem
                //    mockTypescriptFs(project, data);
                //}

                var cli = require('npm');
                cli.load(conf, (err) => {
                    // handle errors
                    if (err) {
                        return reject(err);
                    }

                    // install module
                    cli.commands[args[0]](args.slice(1), (er, data) => {
                        if (er) {
                            return reject(er);
                        }
                        // log errors or data
                        resolve(data);
                    });

                    cli.on('log', (message) => {
                        // log installation progress
                        console.log(message);
                    });
                });
            });
        })
        .then((r => {
            process.chdir(previous);

            return r;
        }));
};
module.exports = npm;


[Function: npm]